In [1]:
#Import Libraries
import pandas as pd
import requests
import json
import os
from pandas.io.json import json_normalize  

In [2]:
#Get API key
from config import appKey

In [3]:
#File Path
file_path=os.path.join("..","Final Output Data","districtRank_data.csv")

#Parameters
param0="&perPage=50"
param1="&appID=59309557"
param2="&appKey="+appKey
param3="&page="

#Base URL
base_url=url="https://api.schooldigger.com/v1.2/rankings/districts/NJ?year=2019"+param0+param1+param2

In [4]:
#Get page count and school count info from header
page_count=28
school_count: 1356
    
#List to store district result set
df_list=[] 

In [5]:
#Loop through all pages
for i in range(1,29):
    
    #Set page # parameter
    param3="&page="+str(i)
    
    #Build URL
    url=base_url+param3
    
    #Get json response
    response = requests.get(url).json()
    
    #Normalize json file
    df = json_normalize(response)
    
    try:
        #
        #Get results from the data frame
        district_set=pd.DataFrame(df["districtList"][0])

        #lets get address details
        district_address = json_normalize(district_set['address']) 

        #Get district address details
        try:
            street=district_address["street"]
        except:
            street=""
        try:
            city=district_address["city"]
        except:
            city=""
        try:
            zipcode=district_address["zip"]
        except:
            zipcode=""
        try:
            lat=district_address["latLong.latitude"]
        except:
            lat=""
        try:
            lng=district_address["latLong.longitude"]
        except:
            lng=""

        #Append columns in school district data set
        district_set["City"]=city
        district_set["Zip Code"]=zipcode
        district_set["Address"]=street
        district_set["Lat"]=lat
        district_set["Lng"]=lng

        #Lets work on school rank history
        school_rank=pd.DataFrame(district_set['rankHistory'][0])

        #Get school ranks - interested in 2018 and 2019 
        try:
            year2018=school_rank["year"].iloc[0]
        except:
            year2018="2018"
        try:
            year2018rank=school_rank["rank"].iloc[0]
        except:
            year2018rank=""     
        try:
            year2018rankof=school_rank["rankOf"].iloc[0]
        except:
            year2018rankof=""
        try:
            year2018rankstar=school_rank["rankStars"].iloc[0]
        except:
            year2018rankstar=""
        try:
            year2018staterank=school_rank["rankStatewidePercentage"].iloc[0]
        except:
            year2018staterank=""
        try:
            year2018rankscore=school_rank["rankScore"].iloc[0]
        except:
            year2018rankscore=""
        try:
            year2019=school_rank["year"].iloc[1]
        except:
            year2019="2019"
        try:
            year2019rank=school_rank["rank"].iloc[1]
        except:
            year2019rank=""
        try:
            year2019rankstar=school_rank["rankOf"].iloc[1]
        except:
            year2019rankstar=""
        try:
            year2019rankof=school_rank["rankStars"].iloc[1]
        except:
            year2019rankof=""
        try:
            year2019staterank=school_rank["rankStatewidePercentage"].iloc[1]
        except:
            year2019staterank=""
        try:
            year2019rankscore=school_rank["rankScore"].iloc[1]
        except:
            year2019rankscore=""

        #Append columns in df4
        #district_set["Year 2018"]=year2018
        district_set["2018 Rank"]=year2018rank
        district_set["2018 Rank of"]=year2018rankof
        district_set["2018 Rank Stars"]=year2018rankstar
        district_set["2018 State Rank"]=year2018staterank
        district_set["2018 Rank Score"]=year2018rankscore
        #district_set["Year 2019"]=year2019
        district_set["2019 Rank"]=year2019rank
        district_set["2019 Rank of"]=year2019rankof
        district_set["2019 Rank Stars"]=year2019rankstar
        district_set["2019 State Rank"]=year2019staterank
        district_set["2019 Rank Score"]=year2019rankscore

        #Get school details
        school_info=pd.DataFrame(district_set['districtYearlyDetails'][0])
        try:
            studentcount_2019=school_info["numberOfStudents"]
        except:
            studentcount_2019=""
            
        district_set["2019 Student Count"]=studentcount_2019

        #Populate Data 
        df_list.append(district_set)
    except:
        continue

In [6]:
#Check length of list -should be 28 sets
len(df_list) #Looks like to sets not returned

13

In [7]:
#Lets merge all data
district_data = pd.concat(df_list,sort=False)

In [8]:
#Check district data columns
district_data.columns

Index(['districtID', 'districtName', 'phone', 'url', 'address', 'hasBoundary',
       'county', 'lowGrade', 'highGrade', 'numberTotalSchools',
       'numberPrimarySchools', 'numberMiddleSchools', 'numberHighSchools',
       'numberAlternativeSchools', 'rankHistory', 'districtYearlyDetails',
       'City', 'Zip Code', 'Address', 'Lat', 'Lng', '2018 Rank',
       '2018 Rank of', '2018 Rank Stars', '2018 State Rank', '2018 Rank Score',
       '2019 Rank', '2019 Rank of', '2019 Rank Stars', '2019 State Rank',
       '2019 Rank Score', '2019 Student Count'],
      dtype='object')

In [9]:
#Drop columns that we extracted data from
district_data=district_data.drop(columns=["phone","url","address","hasBoundary","county","numberPrimarySchools",
                                             "numberMiddleSchools","numberHighSchools","numberAlternativeSchools",                                           
                                             "rankHistory","districtYearlyDetails"],axis=1)

In [10]:
#Rename columns
district_data = district_data.rename(columns ={"districtID": "Dist ID","districtName":"Dist Name",
                           "lowGrade":"Low Grade","highGrade":"High Grade","numberTotalSchools":"School Count"})

In [11]:
#Reorder column names
district_data=district_data[['Dist ID', 'Dist Name', 'City', 'Zip Code', 'Address', 'Lat', 'Lng',  
                                     '2018 Rank', '2018 Rank of', '2018 Rank Stars', '2018 State Rank','2018 Rank Score',
                                     '2019 Rank','2019 Rank of', '2019 Rank Stars','2019 State Rank', '2019 Rank Score',
                                     '2019 Student Count', 'Low Grade', 'High Grade', 'School Count']]

In [12]:
#Display sample data
district_data.head(2)

,Dist ID,Dist Name,City,Zip Code,Address,Lat,Lng,2018 Rank,2018 Rank of,2018 Rank Stars,...,2018 Rank Score,2019 Rank,2019 Rank of,2019 Rank Stars,2019 State Rank,2019 Rank Score,2019 Student Count,Low Grade,High Grade,School Count
0,3409930,Mendham Township School District,Brookside,07926,18 West Main Street,,,1,618,5,...,0.994291,2,5,611,99.67,0.990884,696.0,PK,8,2
1,3414370,Rumson Borough School District,Rumson,07760,60 Forrest Avenue,,,1,618,5,...,0.994291,2,5,611,99.67,0.990884,NaN,PK,8,2


In [13]:
#Chek if data is loaded sucessfully
district_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 618 entries, 0 to 17
Data columns (total 21 columns):
Dist ID               618 non-null object
Dist Name             618 non-null object
City                  618 non-null object
Zip Code              618 non-null object
Address               618 non-null object
Lat                   618 non-null object
Lng                   618 non-null object
2018 Rank             618 non-null int64
2018 Rank of          618 non-null int64
2018 Rank Stars       618 non-null int64
2018 State Rank       618 non-null float64
2018 Rank Score       618 non-null float64
2019 Rank             618 non-null object
2019 Rank of          618 non-null object
2019 Rank Stars       618 non-null object
2019 State Rank       618 non-null object
2019 Rank Score       618 non-null object
2019 Student Count    13 non-null float64
Low Grade             618 non-null object
High Grade            618 non-null object
School Count          618 non-null int64
dtypes: float64(

In [14]:
#Export data to csv
district_data.to_csv(file_path)